# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the training data and do a train/validation split.
2. Preprocess data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate performance of final neural network on testing data.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

In [240]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import math


## Load the Data

Start by importing the data from the pickle file.

In [241]:
with open('train.p', 'rb') as f:
    data = pickle.load(f)

## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [242]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], random_state=0, test_size=0.33)


In [243]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Preprocess the Data

Now that you've loaded the training data, preprocess the data such that it's in the range between -0.5 and 0.5.

In [244]:
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_train = X_train / 255.0 - 0.5
X_val = X_val / 255.0 - 0.5

In [245]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [246]:
# TODO: Build a two-layer feedforward neural network with Keras here.
from keras.models import Sequential
from keras.layers import Dense, Input, Activation

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(32*32*3,)))
model.add(Dense(43, activation='softmax'))

In [247]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
dense_layers = []
for l in model.layers:
    if type(l) == Dense:
        dense_layers.append(l)
assert(len(dense_layers) == 2), "There should be 2 Dense layers."
d1 = dense_layers[0]
d2 = dense_layers[1]
assert(d1.input_shape == (None, 3072))
assert(d1.output_shape == (None, 128))
assert(d2.input_shape == (None, 128))
assert(d2.output_shape == (None, 43))

last_layer = model.layers[-1]
assert(last_layer.activation.__name__ == 'softmax'), "Last layer should be softmax activation, is {}.".format(last_layer.activation.__name__)

In [248]:
# Debugging
for l in model.layers:
    print(l.name, l.input_shape, l.output_shape, l.activation)

dense_39 (None, 3072) (None, 128) <function relu at 0x144617950>
dense_40 (None, 128) (None, 43) <function softmax at 0x144617730>


## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [249]:
from keras.utils import np_utils

Y_train = np_utils.to_categorical(y_train, 43)
Y_val = np_utils.to_categorical(y_val, 43)

X_train_flat = X_train.reshape(-1, 32*32*3)
X_val_flat = X_val.reshape(-1, 32*32*3)

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train_flat, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val_flat, Y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_39 (Dense)                 (None, 128)           393344      dense_input_29[0][0]             
____________________________________________________________________________________________________
dense_40 (Dense)                 (None, 43)            5547        dense_39[0][0]                   
Total params: 398891
____________________________________________________________________________________________________
Train on 26270 samples, validate on 12939 samples
Epoch 1/20
26270/26270 [==============================] - 3s - loss: 1.8935 - acc: 0.5104 - val_loss: 1.1476 - val_acc: 0.6849
Epoch 2/20
26270/26270 [==============================] - 3s - loss: 0.9281 - acc: 0.7598 - val_loss: 0.7406 - val_acc: 0.8236
Epoch 3/20
26270/26270 [==============================] - 3s - loss: 

In [319]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
#assert(history.history['acc'][-1] > 0.92), "The training accuracy was: %.3f" % history.history['acc'][-1])
#assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1])

**Validation Accuracy**: 91.4%

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [267]:
from keras.layers import Convolution2D, Flatten

model2 = Sequential()
model2.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(32, 32, 3))) #outputs (32,32,32)
model2.add(Flatten()) # flattens dimensions to 32*32*32 or 32768
model2.add(Dense(128, activation='relu', input_shape=(32768,)))
model2.add(Dense(43, activation='softmax'))

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model2.fit(X_train, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val, Y_val))

Train on 26270 samples, validate on 12939 samples
Epoch 1/20
26270/26270 [==============================] - 37s - loss: 0.9961 - acc: 0.7449 - val_loss: 0.3704 - val_acc: 0.9094
Epoch 2/20
26270/26270 [==============================] - 36s - loss: 0.2436 - acc: 0.9407 - val_loss: 0.2293 - val_acc: 0.9450
Epoch 3/20
26270/26270 [==============================] - 35s - loss: 0.1301 - acc: 0.9676 - val_loss: 0.2188 - val_acc: 0.9466
Epoch 4/20
26270/26270 [==============================] - 32s - loss: 0.0898 - acc: 0.9774 - val_loss: 0.2008 - val_acc: 0.9498
Epoch 5/20
26270/26270 [==============================] - 32s - loss: 0.0652 - acc: 0.9834 - val_loss: 0.2053 - val_acc: 0.9519
Epoch 6/20
26270/26270 [==============================] - 35s - loss: 0.0432 - acc: 0.9891 - val_loss: 0.1824 - val_acc: 0.9593
Epoch 7/20
26270/26270 [==============================] - 34s - loss: 0.0460 - acc: 0.9887 - val_loss: 0.2215 - val_acc: 0.9539
Epoch 8/20
26270/26270 [==============================

In [317]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 96.7%

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [310]:
from keras.layers.pooling import MaxPooling2D #2d for spatial data

model3 = Sequential()
model3.add(Convolution2D(32, 3, 3, border_mode = 'valid', input_shape=(32, 32, 3))) #output shape None, 30, 30, 32
model3.add(MaxPooling2D(pool_size = (2,2))) # None, 15, 15, 32
model3.add(Flatten()) # None, 7200
model3.add(Dense(128, activation='relu')) # None, 128
model3.add(Dense(43, activation='softmax')) # None, 43

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model3.fit(X_train, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val, Y_val))




Train on 26270 samples, validate on 12939 samples
Epoch 1/20
26270/26270 [==============================] - 23s - loss: 1.4285 - acc: 0.6271 - val_loss: 0.5587 - val_acc: 0.8611
Epoch 2/20
26270/26270 [==============================] - 23s - loss: 0.3601 - acc: 0.9140 - val_loss: 0.2807 - val_acc: 0.9356
Epoch 3/20
26270/26270 [==============================] - 22s - loss: 0.1861 - acc: 0.9600 - val_loss: 0.2024 - val_acc: 0.9536
Epoch 4/20
26270/26270 [==============================] - 21s - loss: 0.1264 - acc: 0.9708 - val_loss: 0.1780 - val_acc: 0.9593
Epoch 5/20
26270/26270 [==============================] - 22s - loss: 0.0848 - acc: 0.9828 - val_loss: 0.1567 - val_acc: 0.9672
Epoch 6/20
26270/26270 [==============================] - 22s - loss: 0.0598 - acc: 0.9891 - val_loss: 0.1472 - val_acc: 0.9692
Epoch 7/20
26270/26270 [==============================] - 23s - loss: 0.0460 - acc: 0.9917 - val_loss: 0.1618 - val_acc: 0.9662
Epoch 8/20
26270/26270 [==============================

In [316]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 98.0%

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [334]:
from keras.layers.core import Dropout

model4 = Sequential()
model4.add(Convolution2D(32, 3, 3, border_mode = 'valid', input_shape=(32, 32, 3))) #output shape None, 30, 30, 32
model4.add(MaxPooling2D(pool_size = (2,2))) # None, 15, 15, 32
model4.add(Dropout(p=0.5)) #None, 15, 15, 32 
model4.add(Activation('relu')) # None, 15, 15, 32
model4.add(Flatten()) # None, 7200
model4.add(Dense(128, activation='relu')) # None, 128
model4.add(Dense(43, activation='softmax')) # None, 43

model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model4.fit(X_train, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val, Y_val))

Train on 26270 samples, validate on 12939 samples
Epoch 1/20
26270/26270 [==============================] - 30s - loss: 1.7513 - acc: 0.5413 - val_loss: 0.7977 - val_acc: 0.7989
Epoch 2/20
26270/26270 [==============================] - 26s - loss: 0.5956 - acc: 0.8428 - val_loss: 0.4022 - val_acc: 0.9080
Epoch 3/20
26270/26270 [==============================] - 26s - loss: 0.3586 - acc: 0.9075 - val_loss: 0.2885 - val_acc: 0.9291
Epoch 4/20
26270/26270 [==============================] - 29s - loss: 0.2595 - acc: 0.9330 - val_loss: 0.2056 - val_acc: 0.9537
Epoch 5/20
26270/26270 [==============================] - 28s - loss: 0.2099 - acc: 0.9453 - val_loss: 0.1853 - val_acc: 0.9536
Epoch 6/20
26270/26270 [==============================] - 27s - loss: 0.1705 - acc: 0.9566 - val_loss: 0.1616 - val_acc: 0.9621
Epoch 7/20
26270/26270 [==============================] - 28s - loss: 0.1506 - acc: 0.9612 - val_loss: 0.1399 - val_acc: 0.9711
Epoch 8/20
26270/26270 [==============================

In [336]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 97.9%

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [344]:
#read data
with open('test.p', 'rb') as f:
    test = pickle.load(f)

#split data
X_test = test['features']
y_test = test['labels']

#normalize
X_test = X_test / 255.0 - 0.5

#one hot encode
y_test = np_utils.to_categorical(y_test, 43)

#evaluate model
model4.evaluate(X_test, y_test)



12630/12630 [==============================] - 4s     


[0.48976742956058805, 0.90894695169285755]

**Testing Accuracy**: 90.8%

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.